In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from sklearn.cluster import KMeans
import csv
import openpyxl

In [ ]:
# ignorar avisos
import warnings
# Suprimir apenas o SettingWithCopyWarning
warnings.filterwarnings('ignore')

In [ ]:
solicitacoes_credito = pd.read_csv('Arquivos/solicitacoescredito.csv')

solicitacoes_credito.head()

In [ ]:

# solicitacoes = solicitacoes_credito[['razaoSocial', 'nomeFantasia', 'cnpjSemTraco', 'margemBrutaAcumulada', 'maiorAtraso','percentualProtestos','primeiraCompra','prazoMedioRecebimentoVendas','titulosEmAberto','valorSolicitado','valorAprovado','dataAprovadoEmComite','ativoCirculante','passivoCirculante','totalAtivo','totalPatrimonioLiquido','endividamento','faturamentoBruto','margemBruta','custos','anoFundacao','intervaloFundacao','capitalSocial','restricoes','scorePontualidade','limiteEmpresaAnaliseCredito'

# clientes = solicitacoes_credito[['cnpjSemTraco','faturamentoBruto', 'intervaloFundacao', 'empresa_MeEppMei', 'endividamento', 'maiorAtraso', 'restricoes','scorePontualidade']]

# clientes = solicitacoes_credito[['faturamentoBruto', 'anoFundacao', 'maiorAtraso', 'restricoes']]

# clientes['TempoDeExistencia'] = 2024 - clientes['anoFundacao']

clientes = solicitacoes_credito[['faturamentoBruto', 'anoFundacao', 'maiorAtraso', 'restricoes']]

clientes


In [ ]:
clientes1 = clientes.drop_duplicates()

display(clientes1)

In [ ]:
clientes2 = clientes1.dropna(subset=['faturamentoBruto'])
clientes2.head()
clientes2


In [ ]:
clientes2 = clientes2.drop(columns=['anoFundacao'])

clientes2


In [ ]:
# clientes3 = clientes1[clientes1['faturamentoBruto'] <= 1]
clientes3 = clientes2
clientes3.head()
# clientes3


In [ ]:
# clientes3.endividamento.value_counts(dropna=False)

# filtra quando nulos, similar a select * from clientes3 where endividamento isnull
clientesNAN = clientes3.loc[clientes3.faturamentoBruto.isnull()]
clientesNAN

In [ ]:
# # Filtra os clientes com scorePontualidade igual a 1
# # teste_clientes2 = clientes3.loc[clientes3['scorePontualidade'] == 1]

# # Encontra o cliente com a menor quantidade de dias de atraso
# cliente_min_atraso = teste_clientes2.loc[teste_clientes2['maiorAtraso'].idxmin()]

# # Encontra o cliente com a maior quantidade de dias de atraso
# cliente_max_atraso = teste_clientes2.loc[teste_clientes2['maiorAtraso'].idxmax()]

# # Exibe os resultados
# print("Cliente com menor atraso:")
# display(cliente_min_atraso[['cnpjSemTraco','maiorAtraso']])

# print("\nCliente com maior atraso:")
# print(cliente_max_atraso[['cnpjSemTraco','maiorAtraso']])

In [ ]:
# clientes3.endividamento = clientes3.endividamento.fillna(0)
# clientes3.loc[pd.isnull(clientes3.endividamento), 'endividamento'] = 0  
# clientes3['anoFundacao'] = clientes3['anoFundacao'].astype(int)

display(clientes3)

# A partir daqui começa a Clusterização
# 
# Referencia: https://www.youtube.com/watch?v=GBSTT5NBE4g

In [ ]:
# Trazendo percentual de pontuação 
# clientes3.scorePontualidade.value_counts() / clientes3.shape[0]

# Trazendo os clintes com score abaixo de 1 
# clientes3[clientes3['scorePontualidade'] < 1]['scorePontualidade'].value_counts()

# Traz quantos valores estão nulos
# clientes3.scorePontualidade.value_counts(dropna=False)

In [ ]:
# transformar todos os dados não numericos em numericos fazendo que as linhas virem colunas
clientes_dummies = pd.get_dummies(clientes3)
clientes_dummies.head()

In [ ]:
# Ele soma todos os que são 1 ou true 
clientes_dummies.sum(axis=1)

In [ ]:
# Dividindo todas as linhas pela soma delas 
X = clientes_dummies.div(clientes_dummies.sum(axis=1), axis='rows')
# X = clientes_dummies.sum(axis=1)
X.head()

In [ ]:
# Agrupamento
km = KMeans(n_clusters=8)
km.fit_predict(X)

In [ ]:
# Descobrindo qual o melhor numero de clusters(cotovelo)
distorcions = []
n_clusters = range(2, 15)
for n_cluster2 in n_clusters:
    print(n_cluster2)
    distorcions.append(KMeans(n_clusters=n_cluster2, max_iter=10_000, n_init=100, random_state=61658).fit(X).inertia_)


In [ ]:
distorcions


In [ ]:
plt.figure(figsize=(15,5))
plt.plot(n_clusters, distorcions)
plt.grid
# Aqui, consideramos o ultimo "cotovelo" que no caso é o 4

In [ ]:
km = KMeans(n_clusters=4, max_iter=10_000, n_init=100, random_state=61658)
X_t = km.fit_predict(X)
X_t


In [ ]:
clientes4 = clientes3
clientes4.loc[:, 'cluster'] = X_t
clientes4


In [ ]:
clientes4.loc[clientes4.cluster==1]

In [ ]:
clientes_dummies.loc[:, 'cluster'] = X_t
clientes_dummies.head()


In [ ]:
# Verifica se X é um DataFrame e converte para array NumPy se necessário
if isinstance(X, pd.DataFrame):
    X = X.to_numpy()

# Ajusta a criação e a previsão do modelo KMeans
km = KMeans(n_clusters=5, max_iter=10_000, n_init=100, random_state=61658)
X_t = km.fit_predict(X)

# Atualiza o DataFrame clientes4 com os clusters
clientes4 = clientes3.copy()
clientes4['cluster'] = X_t

# Cria o gráfico de dispersão
plt.figure(figsize=(10, 6))

# Plota os dados com os clusters
plt.scatter(X[:, 0], X[:, 1], c=X_t, s=50, cmap='viridis', alpha=0.6, edgecolors='w')

# Plota os centros dos clusters
centers = km.cluster_centers_
plt.scatter(centers[:, 0], centers[:, 1], c='black', s=200, alpha=0.75, marker='x')

plt.title('Gráfico de Dispersão com Clusters')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.colorbar(label='Cluster')

plt.grid(True)
plt.show()

In [ ]:
# # separa clientes por cluster 
# saida = []

# # separa clientes por cluster 
# for col in clientes4.drop('clus', axis=1).columns:
#     for cl in np.sort(clientes4.clus.unique()):
#         if clientes4.dtypes[col] == object:
#             # vc = 100*clientes4.loc[clientes4.clus==cl,col].value_counts() / (clientes4.clus==cl).sum()
#             vc = clientes4.loc[clientes4.clus==cl,col].value_counts() / (clientes4.clus==cl).sum()
#             for cat,cnt in vc.reset_index().values:
#                 # print(f'{cl:d};{col};{str(cat):s};{cnt:5.2f}%'.replace(',', '.'))
#                 print(f'{cl:d};{col};{str(cat):s};{cnt:5.2f}'.replace(',', '.'))
#                 saida.append([cl, col, str(cat), f'{cnt:5.2f}'])
#         else:
#             # print(f'{cl:d};{col};{clientes4.loc[clientes4.clus==cl,col].mean():.2f}%'.replace(',', '.'))
#             print(f'{cl:d};{col};{clientes4.loc[clientes4.clus==cl,col].mean():.2f}'.replace(',', '.'))
#             saida.append([cl, col, str(cat), f'{cnt:5.2f}'])

# for col in clientes4.drop('clus', axis=1).columns:
#     if clientes4.dtypes[col] == object:
#         # vc = 100*clientes4.loc[:,col].value_counts() / (clientes4.shape[0])
#         vc = clientes4.loc[:,col].value_counts() / (clientes4.shape[0])
#         for cat,cnt in vc.reset_index().values:
#             # print(f'All;{col};{str(cat):s};{cnt:5.2f}%'.replace(',', '.')) 
#             print(f'All;{col};{str(cat):s};{cnt:5.2f}'.replace(',', '.')) 
#             saida.append([cl, col, str(cat), f'{cnt:5.2f}'])
#     else:
#         # print(f'All;{col};{clientes4.loc[:,col].mean():.2f}%'.replace(',', '.'))
#         print(f'All;{col};{clientes4.loc[:,col].mean():.2f}'.replace(',', '.'))
#         saida.append([cl, col, str(cat), f'{cnt:5.2f}'])


Alguns tratamentos

In [ ]:
clienteFinal = clientes4

In [ ]:
# clientes4['dividas'] = clientes4['endividamento'] < 1
# clienteFinal.loc[:, 'endividamento_ativo'] = clienteFinal['endividamento'] < 1

# Adiciona uma nova coluna 'condicao' que será True se 'endividamento' for maior que 'faturamento', caso contrário False
# clienteFinal['DividaMaiorQFaturamento'] = clienteFinal['endividamento'] > clienteFinal['faturamentoBruto']

clienteFinal.head()


In [ ]:
# clienteFinal.loc[clienteFinal.endividamento_ativo==False]

Analisar com gráficos para entender quais variaveis são importantes

In [ ]:
# Salvando em Excel
df_output = pd.DataFrame(clientes4, columns=['Cluster', 'Campo', 'Category/Value', 'Percentage/Mean'])

clienteFinal.to_excel('saida.xlsx', index=False)

In [ ]:
# # Salvando em CSV 

# with open('saida.csv', mode='w', newline='') as file:
#     writer = csv.writer(file)
#     writer.writerow(['Cluster', 'Campo', 'Category/Value', 'Percentage/Mean'])
#     writer.writerows(clientes4)

In [ ]:
# # Função para substituir valores com base no tipo de dado
# def substituir_valor(valor):
#     if isinstance(valor, str):
#         return '-'  # Substitui por '-' se for string
#     elif isinstance(valor, (int, float)):
#         return 0  # Substitui por 0 se for numérico
#     else:
#         return valor

# # Aplicar a função no DataFrame inteiro
# solicitacoes1 = solicitacoes_distinct.applymap(substituir_valor)

In [ ]:
solicitacoes_credito.head()

# %%
teste_clientes = solicitacoes_credito[['scorePontualidade', 'maiorAtraso']].groupby('maiorAtraso').max()
# teste_clientes = solicitacoes_credito[['scorePontualidade', 'maiorAtraso']]
teste_clientes2 = teste_clientes.drop_duplicates()

teste_clientes2=teste_clientes2.loc[teste_clientes2.scorePontualidade==1]

min_maiorAtraso = teste_clientes2.get('maiorAtraso', pd.Series([])).min()
max_maiorAtraso = teste_clientes2.get('maiorAtraso', pd.Series([])).max()


print(max_maiorAtraso, min_maiorAtraso)
teste_clientes.describe()
teste_clientes2